In [1]:
import requests
import pandas as pd
import json
import getpass

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

!pip install --ignore-installed sddk
import sddk

  Using cached https://files.pythonhosted.org/packages/b3/50/c7687fbe78679cc743ea71acb21b67db119890a08dde2981d54db0b74470/sddk-0.1.2-py3-none-any.whl


In [2]:
### produce a dataframe for testing:
df = pd.DataFrame([("a1", "b1", "c1"), ("a2", "b2", "c2")], columns=["a", "b", "c"]) 
df

,a,b,c
0,a1,b1,c1
1,a2,b2,c2


# personal HTTP endpoint session configuration

In [0]:
### to configure HTTP endpoint (variable "sddk_url") for a personal session, leave the function parameter empty
s, sddk_url = sddk.configure_session_and_url()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
personal connection established
endpoint variable has been configured to: https://sciencedata.dk/files/


In [0]:
s.get(sddk_url).ok

True

In [0]:
s.put(sddk_url + "testfile.txt", data="textfile content")

<Response [201]>

# shared folder HTTP endpoint session configuration

In [3]:
# AS FOLDER OWNER

### configure a session with endpoint in the shared folder
### (login as group owner)
s, sddk_url = sddk.configure_session_and_url("TEST_shared_folder", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/TEST_shared_folder/


In [0]:
### put it to the shared folder at sciencedata 
s.put(sddk_url + "df.json", data=df.to_json())

<Response [204]>

In [5]:
# AS ORDINARY USER (case 1)

### configure a session with endpoint in the shared folder
### (login as ordinary user)
s, sddk_url = sddk.configure_session_and_url("documents", "kase@zcu.cz")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/kase%40zcu.cz/documents/


In [0]:
### put it to the shared folder at sciencedata 
s.put(sddk_url + "df.json", data=df.to_json())


In [7]:
# AS ORDINARY USER (case 2)

### configure a session with endpoint in the shared folder
### (login as ordinary user)
s, sddk_url = sddk.configure_session_and_url("SNA", "209026@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://silo1.sciencedata.dk/sharingout/209026%40au.dk/SNA/


In [82]:
s.put(sddk_url + "df_after_redirection.json", data=df.to_json())

True

# Functions under development... 

In [0]:
### here are the full-fledged the functions contained in the package for further development
### you can use them directly from here
### to use the package version, use "sddk.configure_session_and_url()"" 

def configure_session_and_url(shared_folder_name=None, owner=None): ### insert group folder name or leave empty for personal root
  '''
  interactively setup your sciencedata.dk homeurl, username and password
  in the case of shared folder, inquire for its owner as well
  check functionality and redirections
  '''
  ### set username and password
  username = input("sciencedata.dk username (format '123456@au.dk'): ")
  password = getpass.getpass("sciencedata.dk password: ")
  ### establish a request session
  s = requests.Session()
  s.auth = (username, password)
  sciencedata_homeurl_alternatives = ["https://sciencedata.dk/","https://silo1.sciencedata.dk/","https://silo2.sciencedata.dk/","https://silo3.sciencedata.dk/","https://silo4.sciencedata.dk/"]
  for sciencedata_homeurl_root in sciencedata_homeurl_alternatives:
    if s.get(sciencedata_homeurl_root + "files/").ok:
      root_folder_url = sciencedata_homeurl_root + "files/"
      ### SETTING FOR SHARED FOLDER - if their name is passed in: 
      if shared_folder_name != None:
        shared_folder_owner_url = root_folder_url + shared_folder_name + "/"
        if s.get(shared_folder_owner_url).ok: ### if you are owner of the shared folder 
          root_folder_url = shared_folder_owner_url ### use the url as the endpoint
          print("connection with shared folder established with you as its owner")
        else: # otherwise use endpoint for "shared with me"
          if owner==None:
            owner = input("\"" + shared_folder_name + "\" owner's username: ") ### in the case Vojtech is folder owner
          shared_folder_member_url = sciencedata_homeurl_root + "sharingin/" + owner + "/" + shared_folder_name + "/" 
          try:
            redirection = s.get(shared_folder_member_url, allow_redirects=False).headers["Location"]
            if redirection != None:
              shared_folder_member_url = redirection + "/"
          except:
            pass
          if s.get(shared_folder_member_url).ok:
            root_folder_url = shared_folder_member_url
            print("connection with shared folder established with you as its ordinary user")
          else:
            print("connection with shared folder failed")
    break
  print("endpoint variable has been configured to: " + root_folder_url)
  return s, root_folder_url

def file_from_object(file_name_and_loc, python_object):
  if s.get(sciencedata_groupurl + file_name_and_loc).ok: ### if there already is a file with the same name
    new_name = input("file with name \"" + file_name_and_loc.rpartition("/")[2] + "\" already exists in given location. Press Enter to overwrite it or enter a different name (without path)")
    if len(new_name) == 0:
      s.put(sciencedata_groupurl + file_name_and_loc, data=json.dumps(python_object))
    else:
      if "/" in new_name: ### if it is a path
        s.put(sciencedata_groupurl + new_name, data=json.dumps(python_object))
      else: 
        s.put(sciencedata_groupurl + file_name_and_loc.rpartition("/")[0] + new_name, data=json.dumps(python_object))
  else:
    s.put(sciencedata_groupurl + file_name_and_loc, data=json.dumps(python_object))

def object_from_file(file_name_and_loc):
  if s.get(sciencedata_groupurl + file_name_and_loc).ok:
    print("file exists")
    try: 
      return json.loads(s.get(sciencedata_groupurl + file_name_and_loc).content) ### if there already is a file with the same name
    except:
      print("file import failed")
  else:
    print("file does not found; check file name and path.")


In [0]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

In [0]:
file_from_object(sciencedata_url + "textfile.txt", "string to write into file")

In [0]:
edh_xml_data_df = pd.DataFrame(s.get(sciencedata_groupurl + "SDAM_data/EDH/edh_xml_data_df.json").json())

In [0]:
edh_xml_data_df.head(5)

,idno_uri,idno_tm,placenames_refs,text,origdate_text,objecttype,commentary
0,http://edh-www.adw.uni-heidelberg.de/edh/insch...,214091,"[http://www.trismegistos.org/place/019860, htt...",In hoc tumulo meserecordia! Christi requiescet...,551 AD – 600 AD,"[Tafel, 257]",\n\n
1,http://edh-www.adw.uni-heidelberg.de/edh/insch...,415668,"[http://www.trismegistos.org/place/016530, htt...",Alae Noricorum Iulius Quintus Vagdaevercusti v...,71 AD – 130 AD,"[Tafel, 257]",\n\n
2,http://edh-www.adw.uni-heidelberg.de/edh/insch...,192497,"[http://www.trismegistos.org/place/029481, htt...",Imperator Caesar Lucius Septimius Severus Pius...,197 AD,"[unbestimmt, 2]",\n Textwiedergabe nach Eck.\n
3,http://edh-www.adw.uni-heidelberg.de/edh/insch...,199108,"[http://www.trismegistos.org/place/015771, htt...",Saturno Meddensi Augusto sacru...,,"[unbestimmt, 2]",\n\n
4,http://edh-www.adw.uni-heidelberg.de/edh/insch...,201034,"[http://www.trismegistos.org/place/015771, htt...",Lucio Naevio Quadratiano legato Augusti pro pr...,193 AD,"[unbestimmt, 2]",\n Datierung: L. Naevius Quadratianus war 193 ...
